In [1]:
%matplotlib inline
import sys, os, glob

from astropy.io import fits, ascii

import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
import astropy.constants as const
from astropy.coordinates import SkyCoord, concatenate
from astropy import table

In [2]:
import scienceplots

plt.style.use([ 'bright', 'science', 'no-latex', 'notebook'])

In [5]:
from pyqsofit.PyQSOFit import QSOFit

ModuleNotFoundError: No module named 'pyqsofit'